In [1]:
import sys
import os
import glob
import re 
import pandas as pd
import numpy as np
from lxml import etree
import requests
import time
from collections import Counter
import shutil

In [2]:
sys.path.append(os.path.abspath("C:/Users/calvotello/Dropbox/MTB/Göttingen/research/"))
sys.path.append(os.path.abspath("./../../"))


In [3]:
from librarian_robot import downloading_datasets, map_classications_systems, tokenize



In [4]:
df = pd.read_parquet("./../data/1980_2019_data_labels.parquet")




In [5]:
df

,medium,title,title_supplement,year,entry_first,author_first_name,author_last_name,author_gnd_id,editor_first_name,editor_last_name,...,74.41@BK_notation,74.28@BK_notation,73.87@BK_notation,74.17@BK_notation,74.15@BK_notation,74.03@BK_notation,73.99@BK_notation,43.63@BK_notation,73.89@BK_notation,43.02@BK_notation
727067974,Aau,Glossario arabo-italiano (lingua moderna),None,1980,3496:09-10-12,Giovanni,Canova,None,None,None,...,0,0,0,0,0,0,0,0,0,0
723420874,Oax,History in the Text 'Quatrevingt-Treize' and t...,None,1980,2045:31-08-12,Sandy,Petrey,None,None,None,...,0,0,0,0,0,0,0,0,0,0
723395780,Oax,Italic and Romance,Linguistic studies in honor of Ernst Pulgram,1980,2045:31-08-12,Herbert J.,Izzo,None,None,None,...,0,0,0,0,0,0,0,0,0,0
607035692,Aau,Din stihurile jupînului François Villon,None,1980,0001:17-08-09,François,Villon,079415202,Alexandru|Romulus|Val,Alexianu|Vulpescu|Munteanu,...,0,0,0,0,0,0,0,0,0,0
388767014,Aau,The @horror of life,None,1980,3054:11-06-04,Roger L.,Williams,1233577956,Roger Lawrence,Williams,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1644730073,Aau,Soluzioni,a practical grammar of contemporary Italian,2019,KNUB:23-11-18,Denise,De Rôme,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1644599279,Aau,Les @grammairiens italiens face à leur langue ...,None,2019,KNUB:22-11-18,Laurent,Vallance,1680803417,None,None,...,0,0,0,0,0,0,0,0,0,0
1644542471,Aau,Romance phonetics and phonology,None,2019,KNUB:21-11-18,None,None,None,Mark|Juana,Gibson|Gil Fernández,...,0,0,0,0,0,0,0,0,0,0
1643642936,Aau,Nationale Identitätskonstruktionen in Argentinien,Pressediskurse in Zeiten der Krise,2019,MAUB:25-10-18,Simone Maria,Mwangi,1663308705,None,None,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df.columns.tolist()[0:20]

['medium',
 'title',
 'title_supplement',
 'year',
 'entry_first',
 'author_first_name',
 'author_last_name',
 'author_gnd_id',
 'editor_first_name',
 'editor_last_name',
 'editor_gnd_id',
 'isbn',
 'ILNs',
 'content_type_ppn',
 'content_type',
 'publisher',
 'language_text',
 'language_original',
 'pages',
 'comment_isbn']

In [7]:
df.columns.tolist()[20:40]

['issn',
 'comment_issn',
 'place_publication',
 'summary',
 'title_continuing_resource',
 'work_ppn',
 'work_info',
 'work_title',
 'expression_ppn',
 'expression_info',
 'expression_title',
 'DDC_notation',
 'DDC_sachgruppe',
 'DDC_grundnotation',
 'BK_ppn',
 'BK_notation',
 'BK_j',
 'RVK_ppn',
 'RVK_notation',
 'RVK_j']

In [8]:
df.columns.tolist()[40:60]

['RVK_k',
 'keyword_ppn',
 'keyword',
 'keyword_loc_ppn',
 'keyword_loc',
 'lcc_notation',
 'klassifikationssystem_system',
 'klassifikationssystem_notation',
 'uri_description',
 'uri',
 'GOK_ppn',
 'GOK_notation',
 'GOK_j',
 'signatur_place',
 'signatur',
 'signatur_date',
 'Abrufzeichen',
 'query',
 'romance_studies',
 'joined_column']

In [9]:
df.title.fillna("", inplace=True)
df.joined_column.fillna("", inplace=True)
df.publisher.fillna("", inplace=True)


# Tokenizer

In [10]:
from transformers import BertTokenizer, AutoTokenizer

In [11]:
bert_multilingual_uncased_tz = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")

In [12]:
for index, row in df.iterrows():
    df.loc[index, "title_tokenized"] = "|".join(bert_multilingual_uncased_tz.tokenize(row["title"]))
    df.loc[index, "joined_column_tokenized"] = "|".join(bert_multilingual_uncased_tz.tokenize(row["joined_column"]))

    df.loc[index, "publisher_tokenized"] = "|".join(bert_multilingual_uncased_tz.tokenize(row["publisher"]))
    


In [13]:
df.to_parquet("./../data/1980_2019_data_labels_tokenized.parquet")

In [14]:
df

,medium,title,title_supplement,year,entry_first,author_first_name,author_last_name,author_gnd_id,editor_first_name,editor_last_name,...,74.17@BK_notation,74.15@BK_notation,74.03@BK_notation,73.99@BK_notation,43.63@BK_notation,73.89@BK_notation,43.02@BK_notation,title_tokenized,joined_column_tokenized,publisher_tokenized
727067974,Aau,Glossario arabo-italiano (lingua moderna),None,1980,3496:09-10-12,Giovanni,Canova,None,None,None,...,0,0,0,0,0,0,0,gl|##ossa|##rio|arabo|-|italiano|(|lingua|mode...,arabisch|||italienisch,[|s|.|n|.|]
723420874,Oax,History in the Text 'Quatrevingt-Treize' and t...,None,1980,2045:31-08-12,Sandy,Petrey,None,None,None,...,0,0,0,0,0,0,0,history|in|the|text|'|quatre|##ving|##t|-|trei...,"the|title|of|this|study|""|history|in|the|text|...",john|benjamin|##s|publishing|company
723395780,Oax,Italic and Romance,Linguistic studies in honor of Ernst Pulgram,1980,2045:31-08-12,Herbert J.,Izzo,None,None,None,...,0,0,0,0,0,0,0,itali|##c|and|romance,linguistic|studies|in|honor|of|ernst|pul|##gra...,john|benjamin|##s|publishing|company
607035692,Aau,Din stihurile jupînului François Villon,None,1980,0001:17-08-09,François,Villon,079415202,Alexandru|Romulus|Val,Alexianu|Vulpescu|Munteanu,...,0,0,0,0,0,0,0,din|sti|##hur|##ile|ju|##pin|##ului|francois|v...,ubersetzung|||rum|##anis|##ch|||ly|##rik,ed|.|univers|||arta|grafica|[|druck|##er|]
388767014,Aau,The @horror of life,None,1980,3054:11-06-04,Roger L.,Williams,1233577956,Roger Lawrence,Williams,...,0,0,0,0,0,0,0,the|@|horror|of|life,sonst|##ige|||franzosisch|||literatur|||pes|##...,wei|##den|##feld|&|nicolson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1644730073,Aau,Soluzioni,a practical grammar of contemporary Italian,2019,KNUB:23-11-18,Denise,De Rôme,None,None,None,...,0,0,0,0,0,0,0,sol|##uzioni,a|practical|grammar|of|contemporary|italian|||...,"routledge|,|taylor|&|francis|group"
1644599279,Aau,Les @grammairiens italiens face à leur langue ...,None,2019,KNUB:22-11-18,Laurent,Vallance,1680803417,None,None,...,0,0,0,0,0,0,0,les|@|gram|##mai|##rien|##s|italiens|face|a|le...,bei|##he|##fte|zur|zeitschrift|fur|romani|##sc...,de|gruyter
1644542471,Aau,Romance phonetics and phonology,None,2019,KNUB:21-11-18,None,None,None,Mark|Juana,Gibson|Gil Fernández,...,0,0,0,0,0,0,0,romance|phone|##tics|and|pho|##nol|##ogy,pho|##nologie|||romani|##sche|sprachen|||phone...,oxford|university|press
1643642936,Aau,Nationale Identitätskonstruktionen in Argentinien,Pressediskurse in Zeiten der Krise,2019,MAUB:25-10-18,Simone Maria,Mwangi,1663308705,None,None,...,0,0,0,0,0,0,0,nationale|identitat|##sko|##nst|##ruk|##tionen...,presse|##disk|##urs|##e|in|zeiten|der|krise|||...,de|gruyter
